Created by Joan-Marc Fisa

- Numerai: [FisaGol](https://numer.ai/fisagol)

- Twitter: [@fisagol](https://twitter.com/fisagol)


In [1]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [2]:
!pip install numerapi

In [3]:

!pip install vecstack;

  Created wheel for vecstack: filename=vecstack-0.4.0-py3-none-any.whl size=19878 sha256=84f95d43bafe5ad534fecdada08fab00d3be48d699043b4abcac342fb7191d28
  Stored in directory: /root/.cache/pip/wheels/28/fe/0c/fe8e43660e3316d7ce204e59a79a72246c0ae9b6c5c79841c8
Successfully built vecstack


In [4]:
import os
import pathlib
import numpy as np
import pandas as pd
import scipy.stats
import tensorflow as tf
import joblib
from tqdm.notebook import tqdm

'''
#################### MDO ###########################

import torch
from torch.nn import Linear
from torch.nn import Sequential
from torch.functional import F
'''

'\n#################### MDO ###########################\n\nimport torch\nfrom torch.nn import Linear\nfrom torch.nn import Sequential\nfrom torch.functional import F\n'

In [5]:
NUMERAI_S3_BUCKET_URL = "https://numerai-public-datasets.s3-us-west-2.amazonaws.com"
#read in the example predictions from local storage
#EXAMPLE_PREDS = 'tournament_predictions.csv'
#or downlod the example predictions from Numerai's S3 bucket:
EXAMPLE_PREDS_URL = NUMERAI_S3_BUCKET_URL + "/latest_numerai_example_predictions_data.csv.xz"
#download the latest tournament data file:
TOURNAMENT_DATA_URL = NUMERAI_S3_BUCKET_URL + "/latest_numerai_tournament_data.csv.xz"
###IMPORTANT! DELETE THE FILE BELOW IF YOU CHANGE MODELS! OTHERWISE, RENAME THE FILE FOR YOUR VARIOUS MODELS###
LM_CACHE_FILE = pathlib.Path("neutralization.cache.joblib")

In [6]:

@tf.function(experimental_relax_shapes=True, experimental_compile=True)
def exposures(x, y):
    x = x - tf.math.reduce_mean(x, axis=0)
    x = x / tf.norm(x, axis=0)
    y = y - tf.math.reduce_mean(y, axis=0)
    y = y / tf.norm(y, axis=0)
    return tf.matmul(x, y, transpose_a=True)

'''
#################### MDO ###########################

def exposures(x, y):
    x = x - x.mean(dim=0)
    x = x / x.norm(dim=0)
    y = y - y.mean(dim=0)
    y = y / y.norm(dim=0)
    return torch.matmul(x.T, y)
'''

'\n#################### MDO ###########################\n\ndef exposures(x, y):\n    x = x - x.mean(dim=0)\n    x = x / x.norm(dim=0)\n    y = y - y.mean(dim=0)\n    y = y / y.norm(dim=0)\n    return torch.matmul(x.T, y)\n'

In [7]:
@tf.function(experimental_relax_shapes=True)
def train_loop_body(model, feats, pred, target_exps):
    with tf.GradientTape() as tape:
        exps = exposures(feats, pred[:, None] - model(feats, training=True))
        loss = tf.reduce_sum(tf.nn.relu(tf.nn.relu(exps) - tf.nn.relu(target_exps)) +
                             tf.nn.relu(tf.nn.relu(-exps) - tf.nn.relu(-target_exps)))
    return loss, tape.gradient(loss, model.trainable_variables)

In [8]:
def train_loop(model, optimizer, feats, pred, target_exps, era):
    for i in range(1000000):
        loss, grads = train_loop_body(model, feats, pred, target_exps)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        if loss < 1e-7:
            break
        if i % 10000 == 0:
            tqdm.write(f'era: {era[3:]} loss: {loss:0.7f}', end='\r')

In [9]:
def reduce_exposure(prediction, features, max_exp, era, weights=None):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(310),
        tf.keras.experimental.LinearModel(use_bias=False),
    ])
    feats = tf.convert_to_tensor(features - 0.5, dtype=tf.float32)
    pred = tf.convert_to_tensor(prediction, dtype=tf.float32)
    if weights is None:
        optimizer = tf.keras.optimizers.Adamax()
        start_exp = exposures(feats, pred[:, None])
        target_exps = tf.clip_by_value(start_exp, -max_exp, max_exp)
        train_loop(model, optimizer, feats, pred, target_exps, era)
    else:
        model.set_weights(weights)
    return pred[:,None] - model(feats), model.get_weights()

'''
###################   MDO    ########################################

def reduce_exposure(prediction, features, max_exp):
    # linear model of features that will be used to partially neutralize predictions
    lin = Linear(features.shape[1],  1, bias=False)
    lin.weight.data.fill_(0.)
    model = Sequential(lin)
    optimizer = torch.optim.Adamax(model.parameters(), lr=1e-4)
    feats = torch.tensor(np.float32(features)-.5)
    pred = torch.tensor(np.float32(prediction))
    start_exp = exposures(feats, pred[:,None])
    # set target exposure for each feature to be <= current exposure
    # if current exposure is less than max_exp, or <= max_exp if  
    # current exposure is > max_exp
    targ_exp = torch.clamp(start_exp, -max_exp, max_exp)

    for i in range(100000):
        optimizer.zero_grad()
        # calculate feature exposures of current linear neutralization
        exps = exposures(feats, pred[:,None]-model(feats))
        # loss is positive when any exposures exceed their target
        loss = (F.relu(F.relu(exps)-F.relu(targ_exp)) + F.relu(F.relu(-exps)-F.relu(-targ_exp))).sum()
        print(f'       loss: {loss:0.7f}', end='\r')
        if loss < 1e-7:
            neutralizer = [p.detach().numpy() for p in model.parameters()]
            neutralized_pred = pred[:,None]-model(feats)
            break
        loss.backward()
        optimizer.step()
    return neutralized_pred, neutralizer
'''

"\n###################   MDO    ########################################\n\ndef reduce_exposure(prediction, features, max_exp):\n    # linear model of features that will be used to partially neutralize predictions\n    lin = Linear(features.shape[1],  1, bias=False)\n    lin.weight.data.fill_(0.)\n    model = Sequential(lin)\n    optimizer = torch.optim.Adamax(model.parameters(), lr=1e-4)\n    feats = torch.tensor(np.float32(features)-.5)\n    pred = torch.tensor(np.float32(prediction))\n    start_exp = exposures(feats, pred[:,None])\n    # set target exposure for each feature to be <= current exposure\n    # if current exposure is less than max_exp, or <= max_exp if  \n    # current exposure is > max_exp\n    targ_exp = torch.clamp(start_exp, -max_exp, max_exp)\n\n    for i in range(100000):\n        optimizer.zero_grad()\n        # calculate feature exposures of current linear neutralization\n        exps = exposures(feats, pred[:,None]-model(feats))\n        # loss is positive when 

In [10]:
def reduce_all_exposures(df, column=["prediction"], neutralizers=None,
                                     normalize=True,
                                     gaussianize=True,
                                     era_col="era",
                                     max_exp=0.1): ###<-----SELECT YOUR MAXIMUM FEATURE EXPOSURE HERE###
    if neutralizers is None:
        neutralizers = [x for x in df.columns if x.startswith("feature")]
    neutralized = []
    if LM_CACHE_FILE.is_file():
        cache = joblib.load(LM_CACHE_FILE)
        # Remove weights for eraX if we'd accidentally saved it in the past.
        cache.pop("eraX", None)
    else:
        cache = {}
    for era in tqdm(df[era_col].unique()):
        tqdm.write(era, end='\r')
        df_era = df[df[era_col] == era]
        scores = df_era[column].values
        exposure_values = df_era[neutralizers].values

        if normalize:
            scores2 = []
            for x in scores.T:
                x = (scipy.stats.rankdata(x, method='ordinal') - .5) / len(x)
                if gaussianize:
                    x = scipy.stats.norm.ppf(x)
                scores2.append(x)
            scores = np.array(scores2)[0]

        scores, weights = reduce_exposure(scores, exposure_values,
                                          max_exp, era, cache.get(era))
        if era not in cache and era != "eraX":
            cache[era] = weights
            joblib.dump(cache, LM_CACHE_FILE)
        scores /= tf.math.reduce_std(scores)
        scores -= tf.reduce_min(scores)
        scores /= tf.reduce_max(scores)
        neutralized.append(scores.numpy())

    predictions = pd.DataFrame(np.concatenate(neutralized),
                               columns=column, index=df.index)
    return predictions

'''
############################   MDO   ######################################

def reduce_all_exposures(df, column, neutralizers=[],
                                     normalize=True,
                                     gaussianize=True,
                                     era_col="era",
                                     max_exp=0.1):
    unique_eras = df[era_col].unique()
    computed = []
    for u in unique_eras:
        print(u, '\r')
        df_era = df[df[era_col] == u]
        scores = df_era[column].values
        exposure_values = df_era[neutralizers].values
        
        if normalize:
            scores2 = []
            for x in scores.T:
                x = (scipy.stats.rankdata(x, method='ordinal') - .5) / len(x)
                if gaussianize:
                    x = scipy.stats.norm.ppf(x)
                scores2.append(x)
            scores = np.array(scores2)[0]

        scores, neut = reduce_exposure(scores, exposure_values, max_exp)

        scores /= scores.std()

        computed.append(scores.detach().numpy())

    return pd.DataFrame(np.concatenate(computed), columns=column, index=df.index)
'''

'\n############################   MDO   ######################################\n\ndef reduce_all_exposures(df, column, neutralizers=[],\n                                     normalize=True,\n                                     gaussianize=True,\n                                     era_col="era",\n                                     max_exp=0.1):\n    unique_eras = df[era_col].unique()\n    computed = []\n    for u in unique_eras:\n        print(u, \'\r\')\n        df_era = df[df[era_col] == u]\n        scores = df_era[column].values\n        exposure_values = df_era[neutralizers].values\n        \n        if normalize:\n            scores2 = []\n            for x in scores.T:\n                x = (scipy.stats.rankdata(x, method=\'ordinal\') - .5) / len(x)\n                if gaussianize:\n                    x = scipy.stats.norm.ppf(x)\n                scores2.append(x)\n            scores = np.array(scores2)[0]\n\n        scores, neut = reduce_exposure(scores, exposure_values, max_

In [11]:
#If CUDA isn't set up properly for Tensorflow, then at least maximize the number of threads available for CPU
if not tf.config.list_physical_devices('GPU'):  # No GPU(s) found
    tf.config.threading.set_inter_op_parallelism_threads(2)
    tf.config.threading.set_intra_op_parallelism_threads(os.cpu_count() // 2)

In [12]:
#read-in or download the example predictions
exp_df = pd.read_csv(EXAMPLE_PREDS_URL, index_col=0)

exp_df.head()

,prediction
id,
n0003aa52cab36c2,0.489186
n000920ed083903f,0.491093
n0038e640522c4a6,0.532746
n004ac94a87dc54b,0.507171
n0052fe97ea0c05f,0.503833


In [13]:
#download the tournament data
tournament_df = pd.read_csv(TOURNAMENT_DATA_URL, index_col=0)
tournament_df.head()

,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,feature_intelligence8,feature_intelligence9,feature_intelligence10,feature_intelligence11,feature_intelligence12,feature_charisma1,feature_charisma2,feature_charisma3,feature_charisma4,feature_charisma5,feature_charisma6,feature_charisma7,feature_charisma8,feature_charisma9,feature_charisma10,feature_charisma11,feature_charisma12,feature_charisma13,feature_charisma14,feature_charisma15,feature_charisma16,feature_charisma17,feature_charisma18,feature_charisma19,feature_charisma20,feature_charisma21,feature_charisma22,feature_charisma23,feature_charisma24,feature_charisma25,feature_charisma26,...,feature_wisdom8,feature_wisdom9,feature_wisdom10,feature_wisdom11,feature_wisdom12,feature_wisdom13,feature_wisdom14,feature_wisdom15,feature_wisdom16,feature_wisdom17,feature_wisdom18,feature_wisdom19,feature_wisdom20,feature_wisdom21,feature_wisdom22,feature_wisdom23,feature_wisdom24,feature_wisdom25,feature_wisdom26,feature_wisdom27,feature_wisdom28,feature_wisdom29,feature_wisdom30,feature_wisdom31,feature_wisdom32,feature_wisdom33,feature_wisdom34,feature_wisdom35,feature_wisdom36,feature_wisdom37,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
n0003aa52cab36c2,era121,validation,0.25,0.75,0.50,0.50,0.0,0.75,0.5,0.25,0.50,0.5,0.25,0.0,0.25,0.50,0.25,0.00,0.25,1.00,1.00,0.25,1.00,1.00,0.25,0.25,0.0,0.50,0.25,0.75,0.00,0.50,0.25,0.25,0.25,0.50,0.00,0.50,1.00,0.25,...,0.00,0.0,0.25,0.5,0.25,0.25,0.00,0.25,0.00,0.25,0.50,0.50,0.50,0.50,0.00,0.25,0.75,0.25,0.25,0.50,0.25,0.00,0.25,0.50,0.25,0.5,0.25,0.25,1.00,0.75,0.75,0.75,1.00,0.75,0.50,0.5,1.0,0.00,0.00,0.25
n000920ed083903f,era121,validation,0.75,0.50,0.75,1.00,0.5,0.00,0.0,0.75,0.25,0.0,0.75,0.5,0.00,0.25,0.50,0.00,1.00,0.25,0.25,1.00,1.00,0.25,0.75,0.00,0.0,0.75,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.25,0.00,1.00,0.25,...,0.50,0.5,0.25,1.0,0.50,0.25,0.00,0.25,0.50,0.25,1.00,0.25,0.00,0.50,0.75,0.75,0.50,1.00,1.00,0.25,0.50,0.25,0.50,0.50,0.50,0.5,0.25,0.25,0.75,0.50,0.50,0.50,0.75,1.00,0.75,0.5,0.5,0.50,0.50,0.50
n0038e640522c4a6,era121,validation,1.00,0.00,0.00,1.00,1.0,1.00,1.0,1.00,0.50,0.5,1.00,1.0,1.00,0.75,0.50,0.50,1.00,1.00,0.50,0.50,0.00,1.00,0.50,1.00,0.5,1.00,0.50,1.00,0.25,1.00,1.00,1.00,0.50,1.00,1.00,0.75,1.00,1.00,...,0.25,0.5,0.00,0.0,0.00,0.25,0.25,0.00,0.50,0.00,0.00,0.00,0.25,0.00,0.25,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,0.75,0.0,0.00,0.25,0.00,0.00,0.00,0.00,0.50,0.25,0.00,0.0,0.5,0.50,0.00,1.00
n004ac94a87dc54b,era121,validation,0.75,1.00,1.00,0.50,0.0,0.00,0.0,0.50,0.75,1.0,0.75,0.0,0.50,0.00,0.50,0.75,0.50,0.75,0.25,0.75,0.25,0.75,0.25,0.75,1.0,0.50,0.50,0.75,0.50,1.00,0.50,0.25,0.75,0.25,0.75,0.25,0.75,0.75,...,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.25,0.00,0.25,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.75,0.00,0.00,0.25,0.25,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.0,0.0,0.25,0.25,0.50
n0052fe97ea0c05f,era121,validation,0.25,0.50,0.50,0.25,1.0,0.50,0.5,0.25,0.25,0.5,0.50,1.0,1.00,1.00,1.00,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.00,0.25,0.0,0.00,0.75,0.25,1.00,0.25,1.00,0.75,0.00,1.00,0.75,0.75,0.75,0.25,...,0.00,0.5,0.50,0.0,0.75,0.50,0.75,0.25,0.25,0.25,0.00,0.25,0.50,0.25,1.00,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.75,1.00,1.0,0.75,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.75,1.0,0.0,0.25,1.00,0.75


In [14]:

tournament_df

,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,feature_intelligence8,feature_intelligence9,feature_intelligence10,feature_intelligence11,feature_intelligence12,feature_charisma1,feature_charisma2,feature_charisma3,feature_charisma4,feature_charisma5,feature_charisma6,feature_charisma7,feature_charisma8,feature_charisma9,feature_charisma10,feature_charisma11,feature_charisma12,feature_charisma13,feature_charisma14,feature_charisma15,feature_charisma16,feature_charisma17,feature_charisma18,feature_charisma19,feature_charisma20,feature_charisma21,feature_charisma22,feature_charisma23,feature_charisma24,feature_charisma25,feature_charisma26,...,feature_wisdom8,feature_wisdom9,feature_wisdom10,feature_wisdom11,feature_wisdom12,feature_wisdom13,feature_wisdom14,feature_wisdom15,feature_wisdom16,feature_wisdom17,feature_wisdom18,feature_wisdom19,feature_wisdom20,feature_wisdom21,feature_wisdom22,feature_wisdom23,feature_wisdom24,feature_wisdom25,feature_wisdom26,feature_wisdom27,feature_wisdom28,feature_wisdom29,feature_wisdom30,feature_wisdom31,feature_wisdom32,feature_wisdom33,feature_wisdom34,feature_wisdom35,feature_wisdom36,feature_wisdom37,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
n0003aa52cab36c2,era121,validation,0.25,0.75,0.50,0.50,0.00,0.75,0.50,0.25,0.50,0.50,0.25,0.00,0.25,0.50,0.25,0.00,0.25,1.00,1.00,0.25,1.00,1.00,0.25,0.25,0.00,0.50,0.25,0.75,0.00,0.50,0.25,0.25,0.25,0.50,0.00,0.50,1.00,0.25,...,0.00,0.00,0.25,0.50,0.25,0.25,0.00,0.25,0.00,0.25,0.50,0.50,0.50,0.50,0.00,0.25,0.75,0.25,0.25,0.50,0.25,0.00,0.25,0.50,0.25,0.5,0.25,0.25,1.00,0.75,0.75,0.75,1.00,0.75,0.50,0.50,1.00,0.00,0.00,0.25
n000920ed083903f,era121,validation,0.75,0.50,0.75,1.00,0.50,0.00,0.00,0.75,0.25,0.00,0.75,0.50,0.00,0.25,0.50,0.00,1.00,0.25,0.25,1.00,1.00,0.25,0.75,0.00,0.00,0.75,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.25,0.00,1.00,0.25,...,0.50,0.50,0.25,1.00,0.50,0.25,0.00,0.25,0.50,0.25,1.00,0.25,0.00,0.50,0.75,0.75,0.50,1.00,1.00,0.25,0.50,0.25,0.50,0.50,0.50,0.5,0.25,0.25,0.75,0.50,0.50,0.50,0.75,1.00,0.75,0.50,0.50,0.50,0.50,0.50
n0038e640522c4a6,era121,validation,1.00,0.00,0.00,1.00,1.00,1.00,1.00,1.00,0.50,0.50,1.00,1.00,1.00,0.75,0.50,0.50,1.00,1.00,0.50,0.50,0.00,1.00,0.50,1.00,0.50,1.00,0.50,1.00,0.25,1.00,1.00,1.00,0.50,1.00,1.00,0.75,1.00,1.00,...,0.25,0.50,0.00,0.00,0.00,0.25,0.25,0.00,0.50,0.00,0.00,0.00,0.25,0.00,0.25,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,0.75,0.0,0.00,0.25,0.00,0.00,0.00,0.00,0.50,0.25,0.00,0.00,0.50,0.50,0.00,1.00
n004ac94a87dc54b,era121,validation,0.75,1.00,1.00,0.50,0.00,0.00,0.00,0.50,0.75,1.00,0.75,0.00,0.50,0.00,0.50,0.75,0.50,0.75,0.25,0.75,0.25,0.75,0.25,0.75,1.00,0.50,0.50,0.75,0.50,1.00,0.50,0.25,0.75,0.25,0.75,0.25,0.75,0.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.25,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.75,0.00,0.00,0.25,0.25,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.00,0.00,0.25,0.25,0.50
n0052fe97ea0c05f,era121,validation,0.25,0.50,0.50,0.25,1.00,0.50,0.50,0.25,0.25,0.50,0.50,1.00,1.00,1.00,1.00,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.00,0.25,0.00,0.00,0.75,0.25,1.00,0.25,1.00,0.75,0.00,1.00,0.75,0.75,0.75,0.25,...,0.00,0.50,0.50,0.00,0.75,0.50,0.75,0.25,0.25,0.25,0.00,0.25,0.50,0.25,1.00,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.75,1.00,1.0,0.75,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.75,1.00,0.00,0.25,1.00,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nffd22980

In [15]:
#merge them together
full_df = pd.merge(tournament_df, exp_df, left_index=True, right_index=True)

In [16]:
#this cell executes the full script above and neutralizes the predictions to achieve a maximum 0.1 Feature Exposure
neutralized_df = reduce_all_exposures(full_df)

  0%|          | 0/352 [00:00<?, ?it/s]

In [17]:
neutralized_df

,prediction
id,
n0003aa52cab36c2,0.407459
n000920ed083903f,0.404174
n0038e640522c4a6,0.656825
n004ac94a87dc54b,0.607468
n0052fe97ea0c05f,0.547859
...,...
nffd229800b5273b,0.547820
nffe79bdb284eebb,0.756454
nfff45e137daf47a,0.425988


In [18]:
#save your prediction file locally
neutralized_df.to_csv("FisaGol_1.csv")

In [19]:
!cp FisaGol_1.csv "drive/My Drive/PHOENIXSIGMA/"